In [29]:
import pandas as pd
import urllib2
from bs4 import BeautifulSoup, Tag, UnicodeDammit
import re
import sys
import os
import time

In [2]:
METADATA_FILE = '../data/data/movies_metadata.csv'
metadata_frame = pd.read_csv(METADATA_FILE)

In [3]:
english_movies = list(metadata_frame[metadata_frame['original_language']=='en'].title.unique())

In [4]:
# male: he,him,his,himself,man,boy,lord,sir,father,grandfather,son,grandson,brother,husband,boyfriend,uncle,nephew
# female: she,her,hers,herself,woman,girl,lady,madam,mother,grandmother,daughter,granddaughter,sister,wife,girlfriend,aunt,niece

gender_patterns = [r"(?i)\bhe\b",
                   r"(?i)\bhim\b",
                   r"(?i)\bhis\b",
                   r"(?i)\bhimself\b",
                   r"(?i)\bman\b",
                   r"(?i)\bboy\b",
                   r"(?i)\blord\b",
                   r"(?i)\bsir\b",
                   r"(?i)\bfather\b",
                   r"(?i)\bgrandfather\b",
                   r"(?i)\bson\b", 
                   r"(?i)\bgrandson\b", 
                   r"(?i)\bbrother\b",
                   r"(?i)\bhusband\b",
                   r"(?i)\bboyfriend\b",
                   r"(?i)\buncle\b",
                   r"(?i)\bnephew\b",
                   r"(?i)\bshe\b",
                   r"(?i)\bher\b",
                   r"(?i)\bhers\b",
                   r"(?i)\bherself\b",
                   r"(?i)\bwoman\b",
                   r"(?i)\bgirl\b",
                   r"(?i)\blady\b",
                   r"(?i)\bmadam\b",
                   r"(?i)\bmother\b",
                   r"(?i)\bgrandmother\b",
                   r"(?i)\bdaughter\b", 
                   r"(?i)\bgranddaughter\b", 
                   r"(?i)\bsister\b",
                   r"(?i)\bwife\b",
                   r"(?i)\bgirlfriend\b",
                   r"(?i)\baunt\b",
                   r"(?i)\bniece\b"]

In [24]:
gender_data = []
completed_movies = []

In [34]:
for i in range(len(english_movies)):
    
    # get movie name and script url
    movie = english_movies[i]
    if movie in completed_movies:
        continue
    if movie.split()[0] == 'The':
        movie_name = '-'.join(movie.split()[1:])+',-The'
    else:
        movie_name = '-'.join(movie.split())
    script_url = 'http://www.imsdb.com/scripts/' + movie_name + '.html'
    
    # get script from IMSDB website
    try:
        html_doc = urllib2.urlopen(script_url).read()
    except:
        continue
    soup = BeautifulSoup(html_doc, 'lxml')
    script = soup.find("pre")
    if script == None:
        #write movie_name in file
        continue
    if script.find("pre"):
        if script.find('pre').text != u'\n':
            script = script.find("pre")
    script_text = u''
    for block in script.descendants:
        if(isinstance(block, Tag)):
            continue
        block = UnicodeDammit(block, soup.original_encoding).unicode_markup
        block = block.strip('\n')
        script_text += block
    
    # create bag of words for gender terms
    bow_vector = [0] * len(gender_patterns)
    for j in range(len(gender_patterns)):
        pattern = gender_patterns[j]
        bow_vector[j] = len(re.findall(pattern, script_text))
    if bow_vector == [0] * len(gender_patterns):
        #write movie_name in file
        continue
    
    # write results
    completed_movies.append(movie)
    gender_data.append([movie, script_url, bow_vector])
    sys.stdout.write('\rCompleted '+str(len(completed_movies))+' of '+str(len(english_movies))+'. ')
    
    # pause for a minute after 8 runs
    if (len(completed_movies)%8 == 0):
        time.sleep(60)

Completed 304 of 30357. 

KeyboardInterrupt: 

In [35]:
gender_dataframe = pd.DataFrame(gender_data, columns=['movie', 'url', 'vector'])

In [36]:
gender_dataframe

,movie,url,vector
0,Toy Story,http://www.imsdb.com/scripts/Toy-Story.html,"[158, 86, 220, 27, 14, 13, 0, 4, 0, 0, 1, 0, 0..."
1,Heat,http://www.imsdb.com/scripts/Heat.html,"[370, 113, 283, 4, 38, 3, 0, 1, 3, 0, 3, 0, 4,..."
2,The American President,http://www.imsdb.com/scripts/American-Presiden...,"[166, 66, 87, 4, 19, 4, 0, 79, 9, 0, 0, 0, 0, ..."
3,Casino,http://www.imsdb.com/scripts/Casino.html,"[566, 270, 325, 19, 63, 7, 0, 12, 2, 0, 11, 0,..."
4,Sense and Sensibility,http://www.imsdb.com/scripts/Sense-and-Sensibi...,"[292, 142, 155, 11, 27, 2, 4, 97, 14, 0, 3, 0,..."
5,Four Rooms,http://www.imsdb.com/scripts/Four-Rooms.html,"[245, 86, 220, 23, 134, 3, 0, 11, 2, 0, 1, 0, ..."
6,Copycat,http://www.imsdb.com/scripts/Copycat.html,"[396, 132, 169, 9, 64, 6, 1, 6, 1, 0, 17, 0, 0..."
7,Assassins,http://www.imsdb.com/scripts/Assassins.html,"[451, 86, 261, 20, 41, 5, 0, 4, 1, 0, 1, 0, 0,..."
8,Leaving Las Vegas,http://www.imsdb.com/scripts/Leaving-Las-Vegas...,"[311, 114, 195, 19, 25, 25, 0, 3, 2, 0, 1, 0, ..."
9,Se7en,http://www.imsdb.com/scripts/Se7en.html,"[585, 148, 472, 25, 124, 8, 6, 4, 18, 0, 7, 0,..."


In [38]:
gender_dataframe.to_csv('data/processed/gender_bow.csv')

IOError: [Errno 2] No such file or directory: 'data/processed/gender_bow.csv'